In [1]:
from kiteconnect import KiteConnect
import urllib.parse
import tqdm
from requests import auth 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os, time
from os.path import expanduser
from selenium.webdriver.chrome.options import Options
import pyotp
from datetime import datetime
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from termcolor import colored
import pdb

import sys
import hmac, base64, struct, hashlib, time
import requests
from datetime import datetime

zerodha_cred = {
    'User':{
        'user_id' : 'RK2267',
        'password': "Mahesh@#2023",
        'api_key' : "pv2830q1vbrhu1eu",   
        'common_ans' : "",
        'api_secret' :'gz426yatawim9xruj2p6e51c40k44r7n',  
        'auth_key': 'TOOAIR75STSUWM67LS3NVUX7B2LGKJ7R'
    },

}
my_api_key=zerodha_cred['User']['api_key']

def get_hotp_token(secret, intervals_no):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", intervals_no)
    h = hmac.new(key, msg, hashlib.sha1).digest()
    o = h[19] & 15
    h = (struct.unpack(">I", h[o:o+4])[0] & 0x7fffffff) % 1000000
    return h

def get_totp_token(secret):
    totp = str(get_hotp_token(secret, intervals_no=int(time.time())//30))
    if len(totp) < 6:
        totp = '0' + totp
    return totp


def getkite(_client):

    user_id = zerodha_cred[_client]['user_id']
    password = zerodha_cred[_client]['password']
    api_key = zerodha_cred[_client]['api_key']
    kite = KiteConnect(api_key=api_key)
    auth_key = zerodha_cred[_client]['auth_key']
    url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 

    try:
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        options.add_argument('--disble-dev-shm-usage')
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        time.sleep(2)
        input_fields = driver.find_elements_by_tag_name('input')
        input_fields[0].send_keys(user_id)
        input_fields[1].send_keys(password)
        input_fields[1].send_keys(Keys.ENTER)
        time.sleep(1)
        two_fa = driver.find_elements_by_tag_name('input')
        common_ans = get_totp_token(auth_key)
        two_fa[0].send_keys(str(common_ans))
        two_fa[0].send_keys(Keys.ENTER)
        time.sleep(1)
        while("request_token=" not in driver.current_url):
            print (driver.current_url)
            driver.save_screenshot('login.png')
            time.sleep(3)
        redirect_url = driver.current_url
        #print('redirectURL-',redirect_url)
        parsed = urllib.parse.urlparse(redirect_url)
        query_dict = dict(urllib.parse.parse_qsl(parsed.query))
        request_token = query_dict['request_token']
        driver.close
        return request_token
    except Exception as e:
        print('closing')
        driver.close()
        
def gen_acc_token():
    global acc_key
    users = ['User'] 
    auth_keys={}
    cur_date=datetime.now().strftime("%y_%m_%d")

    for _client in users:
        api_key = zerodha_cred[_client]['api_key']
        api_secret = zerodha_cred[_client]['api_secret']

        req_token=getkite(_client)  
        url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
        print ('url-',url)
        
        kite = KiteConnect(api_key=api_key)
        
        data = kite.generate_session(req_token, api_secret=api_secret)
        
        access_token=data["access_token"]
        print(data['user_name'],'has successfully signed in.')
        acc_key=access_token
        print('Access_token->',access_token)
        auth_keys[_client] = access_token

        auth_keys['date'] = cur_date
        telegram_bot_sendtext("*Auth keys extracted!*\n\nUser = \t\t" + _client   +  "\nAuth Key = \t" + auth_keys[_client] +  "\nDate = \t" + auth_keys['date'])

if __name__ == '__main__':
    try:
        gen_acc_token()
        acc_key=""
    except Exception as e:
        print('Happy Trading !')

url- https://kite.trade/connect/login?v=3&api_key=pv2830q1vbrhu1eu
Kakal Krishna Rao has successfully signed in.
Access_token-> t6DnO3wFHkkkuvod54WLKAzzzpaqQXbe
Happy Trading !


### Data Section

In [2]:
from datetime import timedelta
import pandas as pd

def history_data(token,interval,start_date,end_date):
    
    global acc_key
    
    data1=['']   
    
    url1="https://api.kite.trade/instruments/historical/"+str(token)+"/"+interval+"?from="+start_date+"&to="+end_date  
    
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    
    res1 = requests.get(url1, headers=HEADERS)

    data1[0] = res1.json()
    data1[0] = data1[0]["data"]["candles"]
    
    data1[0]=pd.DataFrame(data1[0])
    
    data1[0] = data1[0].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    return data1

# TOKENS

instruments={3329: 'ABB', 5633: 'ACC', 6401: 'ADANIENT', 3861249: 'ADANIPORTS', 325121: 'AMBUJACEM', 40193: 'APOLLOHOSP',
         54273: 'ASHOKLEY', 60417: 'ASIANPAINT', 5436929: 'AUBANK', 70401: 'AUROPHARMA', 1510401: 'AXISBANK',
         4267265: 'BAJAJ-AUTO', 4268801: 'BAJAJFINSV', 81153: 'BAJFINANCE', 579329: 'BANDHANBNK', 1195009: 'BANKBARODA',
         98049: 'BEL', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2911489: 'BIOCON', 134657: 'BPCL',
         140033: 'BRITANNIA', 2763265: 'CANBK', 177665: 'CIPLA', 5215745: 'COALINDIA', 2955009: 'COFORGE', 1215745: 'CONCOR',
         486657: 'CUMMINSIND', 197633: 'DABUR', 2800641: 'DIVISLAB', 5552641: 'DIXON', 3771393: 'DLF', 225537: 'DRREDDY',
         232961: 'EICHERMOT', 261889: 'FEDERALBNK', 1207553: 'GAIL', 2585345: 'GODREJCP', 315393: 'GRASIM', 589569: 'HAL',
         1850625: 'HCLTECH', 340481: 'HDFC', 341249: 'HDFCBANK', 119553: 'HDFCLIFE', 345089: 'HEROMOTOCO', 348929: 'HINDALCO',
         359937: 'HINDPETRO', 356865: 'HINDUNILVR', 7712001: 'IBULHSGFIN', 1270529: 'ICICIBANK', 2863105: 'IDFCFIRSTB',
         387073: 'INDHOTEL', 2865921: 'INDIGO', 1346049: 'INDUSINDBK', 7458561: 'INDUSTOWER', 408065: 'INFY', 415745: 'IOC',
         3484417: 'IRCTC', 424961: 'ITC', 1723649: 'JINDALSTEL', 3001089: 'JSWSTEEL', 4632577: 'JUBLFOOD', 492033: 'KOTAKBANK',
         2939649: 'LT', 4561409: 'LTIM', 4752385: 'LTTS', 519937: 'M&M', 3400961: 'M&MFIN', 2815745: 'MARUTI',
         1076225: 'MOTHERSON', 1152769: 'MPHASIS', 582913: 'MRF', 1629185: 'NATIONALUM', 3520257: 'NAUKRI',
         4598529: 'NESTLEIND', 3924993: 'NMDC', 2977281: 'NTPC', 633601: 'ONGC', 3689729: 'PAGEIND', 617473: 'PEL',
         4701441: 'PERSISTENT', 3660545: 'PFC', 681985: 'PIDILITIND', 6191105: 'PIIND', 2730497: 'PNB', 2455041: 'POLYCAB',
         3834113: 'POWERGRID', 3365633: 'PVR', 4708097: 'RBLBANK', 3930881: 'RECLTD', 738561: 'RELIANCE', 758529: 'SAIL',
         5582849: 'SBILIFE', 779521: 'SBIN', 794369: 'SHREECEM', 806401: 'SIEMENS', 837889: 'SRF', 857857: 'SUNPHARMA',
         871681: 'TATACHEM', 878593: 'TATACONSUM', 884737: 'TATAMOTORS', 877057: 'TATAPOWER', 895745: 'TATASTEEL',
         2953217: 'TCS', 3465729: 'TECHM', 897537: 'TITAN', 502785: 'TRENT', 2170625: 'TVSMOTOR', 2952193: 'ULTRACEMCO',
         2889473: 'UPL', 784129: 'VEDL', 951809: 'VOLTAS', 969473: 'WIPRO', 975873: 'ZEEL'}


tokens = list(instruments.keys())

# GET DAILY DATA


DailyData={}
for tk in tokens:
    DailyData[tk]= history_data(tk,'day','2023-02-01','2023-02-28')[0]
    
    
# GET HOURLY DATA

HourlyData={}
for tk in tokens:
    HourlyData[tk]= history_data(tk,'hour','2023-02-01','2023-02-28')[0]
    
    

# PREV MONTH DAILY DATA FOR CPR
PrevMonthData={}
for tk in tokens:
    PrevMonthData[tk]=history_data(tk,'day','2023-01-01','2023-01-31')[0]

In [3]:
## Weekly Data

def create_increasing_lists(lst):
    def remove_second_element(lst):
        return [[arr[0] for arr in sublist] for sublist in lst]
    arr = np.array([item[1] for item in lst])
    diffs = np.diff(arr)
    split_indices = np.where(diffs <= 0)[0] + 1
    new_lists = np.split(np.array(lst), split_indices)
    return remove_second_element([list(sublist) for sublist in new_lists if len(sublist) > 0])

def getHourlyHighLow(tk,dates):
    df=HourlyData[tk] 
    dx=df[(pd.to_datetime(dates[0])<= pd.to_datetime(df.Time) ) & (pd.to_datetime(df.Time)<= pd.to_datetime(dates[-1])) ]
    max_close= dx.Close.max()
    min_close= dx.Close.min() 
    return [max_close,min_close]

def getCPR(df):
    
    close = df.iloc[-1]['Close']      
    high = max(df['High'])            
    low = min(df['Low'])              
    Pivot = (high + low + close) / 3
    BC = (high + low) / 2
    TC = (Pivot - BC) + Pivot
    
    return [ BC, Pivot , TC]

def getWeekly(tk,df):
    arr=[]
    for day in range(len(df)):
        arr.append([df.Time.iloc[day],pd.to_datetime(df.Time.iloc[day]).weekday()])
    wk=create_increasing_lists(arr)
    res={}
    for w in range(len(wk)):
        res[f'WEEK_{w+1}']={}
        res[f'WEEK_{w+1}']['Days']=wk[w]
        k=getHourlyHighLow(tk,wk[w])
        res[f'WEEK_{w+1}']['HighClose']=k[0]
        res[f'WEEK_{w+1}']['LowClose']=k[1]
        cpr=getCPR(PrevMonthData[tk])
        res[f'WEEK_{w+1}']['BC']=cpr[0]
        res[f'WEEK_{w+1}']['Pivot']=cpr[1]
        res[f'WEEK_{w+1}']['TC']=cpr[2]
        
    return res

WeeklyData={}
for tk in tokens:
    WeeklyData[tk]=getWeekly(tk,DailyData[tk])


In [4]:
WeeklyData[5633]

{'WEEK_1': {'Days': ['2023-02-01T00:00:00+0530',
   '2023-02-02T00:00:00+0530',
   '2023-02-03T00:00:00+0530'],
  'HighClose': 1947.7,
  'LowClose': 1817.85,
  'BC': 2110.3,
  'Pivot': 2062.9666666666667,
  'TC': 2015.6333333333332},
 'WEEK_2': {'Days': ['2023-02-06T00:00:00+0530',
   '2023-02-07T00:00:00+0530',
   '2023-02-08T00:00:00+0530',
   '2023-02-09T00:00:00+0530',
   '2023-02-10T00:00:00+0530'],
  'HighClose': 2028.35,
  'LowClose': 1906.85,
  'BC': 2110.3,
  'Pivot': 2062.9666666666667,
  'TC': 2015.6333333333332},
 'WEEK_3': {'Days': ['2023-02-13T00:00:00+0530',
   '2023-02-14T00:00:00+0530',
   '2023-02-15T00:00:00+0530',
   '2023-02-16T00:00:00+0530',
   '2023-02-17T00:00:00+0530'],
  'HighClose': 1869.3,
  'LowClose': 1787.85,
  'BC': 2110.3,
  'Pivot': 2062.9666666666667,
  'TC': 2015.6333333333332},
 'WEEK_4': {'Days': ['2023-02-20T00:00:00+0530',
   '2023-02-21T00:00:00+0530',
   '2023-02-22T00:00:00+0530',
   '2023-02-23T00:00:00+0530',
   '2023-02-24T00:00:00+0530'],

In [7]:
from termcolor import colored


for tk in tokens: 
    
    def Change2Search(time):
        return time[:10]+'T00:00:00+0530'
    
    def Change2Display(time):
        return pd.to_datetime(time).strftime(format = "%Y-%m-%d %H:%M:%S")
    
    print(f'------------   {instruments[tk]} DETAILS ----------------------')
    
    df= HourlyData[tk]
    config=WeeklyData[tk]
    # week 2 + week 3
    dates=config['WEEK_2']['Days']+config['WEEK_3']['Days']
    dx=df[(pd.to_datetime(dates[0])<= pd.to_datetime(df.Time) ) & (pd.to_datetime(df.Time)<= pd.to_datetime(dates[-1]))]

    for day in range(len(dx)):
        
        time= Change2Search(dx.Time.iloc[day])
        dtime= Change2Display(dx.Time.iloc[day])
        close = float(dx.Close.iloc[day])
        idx=0
        for j in range(1,4):
            if time in config[f'WEEK_{j}']['Days']:
                idx=j
        prev_week_high=float(config[f'WEEK_{idx-1}']['HighClose'])
        prev_week_low= float(config[f'WEEK_{idx-1}']['LowClose'])
        pivot=float(config[f'WEEK_{idx-1}']['Pivot'])
        tc= float(config[f'WEEK_{idx-1}']['TC'])
        bc= float(config[f'WEEK_{idx-1}']['BC'])
        if close> prev_week_high and close > max(tc,bc):
            
            
            ## Details   
            print(colored('CE CONDITION TRIGGERED !','green'))
            print(f' Instrument : {instruments[tk]}')
            print(f' DateTime : {dtime}')
            print(f' Close Price : {close}')
            print(f' PrevWeek High : {prev_week_high} ')
            print(f' PrevWeek Low : {prev_week_low} ')
            print(f' CPR Pivot : {pivot}')
            print(f' CPR TC  : {tc}')
            print(f' CPR BC : {bc}')
            print('------------------------------------------')
            
            
            ## Find Nearest Strike Price
                # Take current close price search for nearest strike price from saved data (gdrive) by string formatting
                # csv filenames 
            
            sp=round(close,-2)
            
            ## Search offer price for sp at this particular time 
            
            #let
            price=100
                
            intial_per=4
            
            on_take_price=price+price*(intial_per)
            
            # Looop over gdrive data to check target and stoplosss
            
            target1= on_take_price + on_take_price * 100
            target2 = on_take_price+ on_take_price*150
            
            sl1= on_take_price - on_take_price*25
            sl2= on_take_price - on_take_price*50
            
            
            
            ## anyway this target and sl changes as per conditions
            

            

            
        elif close < prev_week_low and close < min(tc,bc):
                        
                
            ## Details   
            print(colored('PE CONDITION TRIGGERED !','red')) 
            print(f' Instrument : {instruments[tk]}')
            print(f' DateTime : {dtime}')
            print(f' Close Price : {close}')
            print(f' PrevWeek High : {prev_week_high} ')
            print(f' PrevWeek Low : {prev_week_low} ')
            print(f' CPR Pivot : {pivot}')
            print(f' CPR TC  : {tc}')
            print(f' CPR BC : {bc} ')
            print('------------------------------------------')
            
            
            ## Same as above for PE operations
            

            

        
        
                

    
    
    
    
    
    

------------   ABB DETAILS ----------------------
CE CONDITION TRIGGERED !
 Instrument : ABB
 DateTime : 2023-02-06 14:15:00
 Close Price : 2933.8
 PrevWeek High : 2901.4 
 PrevWeek Low : 2773.85 
 CPR Pivot : 2827.0
 CPR TC  : 2833.25
 CPR BC : 2820.75
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : ABB
 DateTime : 2023-02-06 15:15:00
 Close Price : 2930.0
 PrevWeek High : 2901.4 
 PrevWeek Low : 2773.85 
 CPR Pivot : 2827.0
 CPR TC  : 2833.25
 CPR BC : 2820.75
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : ABB
 DateTime : 2023-02-07 09:15:00
 Close Price : 2917.9
 PrevWeek High : 2901.4 
 PrevWeek Low : 2773.85 
 CPR Pivot : 2827.0
 CPR TC  : 2833.25
 CPR BC : 2820.75
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : ABB
 DateTime : 2023-02-07 10:15:00
 Close Price : 2922.6
 PrevWeek High : 2901.4 
 PrevWeek Low : 2773.85 
 CPR Pivot : 2827.0
 CPR TC  : 2833.25
 CPR BC : 2820.75
-----

CE CONDITION TRIGGERED !
 Instrument : AUROPHARMA
 DateTime : 2023-02-14 10:15:00
 Close Price : 465.95
 PrevWeek High : 446.0 
 PrevWeek Low : 411.65 
 CPR Pivot : 423.6166666666666
 CPR TC  : 415.8833333333332
 CPR BC : 431.35
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : AUROPHARMA
 DateTime : 2023-02-14 11:15:00
 Close Price : 472.5
 PrevWeek High : 446.0 
 PrevWeek Low : 411.65 
 CPR Pivot : 423.6166666666666
 CPR TC  : 415.8833333333332
 CPR BC : 431.35
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : AUROPHARMA
 DateTime : 2023-02-14 12:15:00
 Close Price : 471.0
 PrevWeek High : 446.0 
 PrevWeek Low : 411.65 
 CPR Pivot : 423.6166666666666
 CPR TC  : 415.8833333333332
 CPR BC : 431.35
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : AUROPHARMA
 DateTime : 2023-02-14 13:15:00
 Close Price : 472.15
 PrevWeek High : 446.0 
 PrevWeek Low : 411.65 
 CPR Pivot : 423.6166666666666
 CP

PE CONDITION TRIGGERED !
 Instrument : BHEL
 DateTime : 2023-02-14 13:15:00
 Close Price : 71.2
 PrevWeek High : 76.3 
 PrevWeek Low : 74.6 
 CPR Pivot : 77.85000000000001
 CPR TC  : 78.12500000000001
 CPR BC : 77.575 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : BHEL
 DateTime : 2023-02-14 14:15:00
 Close Price : 71.0
 PrevWeek High : 76.3 
 PrevWeek Low : 74.6 
 CPR Pivot : 77.85000000000001
 CPR TC  : 78.12500000000001
 CPR BC : 77.575 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : BHEL
 DateTime : 2023-02-14 15:15:00
 Close Price : 71.05
 PrevWeek High : 76.3 
 PrevWeek Low : 74.6 
 CPR Pivot : 77.85000000000001
 CPR TC  : 78.12500000000001
 CPR BC : 77.575 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : BHEL
 DateTime : 2023-02-15 09:15:00
 Close Price : 70.6
 PrevWeek High : 76.3 
 PrevWeek Low : 74.6 
 CPR Pivot : 77.85000000000001
 CPR TC  : 78.12500000000001
 CPR BC : 77

CE CONDITION TRIGGERED !
 Instrument : CUMMINSIND
 DateTime : 2023-02-15 09:15:00
 Close Price : 1600.0
 PrevWeek High : 1595.0 
 PrevWeek Low : 1433.5 
 CPR Pivot : 1437.3999999999999
 CPR TC  : 1436.5999999999997
 CPR BC : 1438.2
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : CUMMINSIND
 DateTime : 2023-02-15 10:15:00
 Close Price : 1601.0
 PrevWeek High : 1595.0 
 PrevWeek Low : 1433.5 
 CPR Pivot : 1437.3999999999999
 CPR TC  : 1436.5999999999997
 CPR BC : 1438.2
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : CUMMINSIND
 DateTime : 2023-02-15 11:15:00
 Close Price : 1595.5
 PrevWeek High : 1595.0 
 PrevWeek Low : 1433.5 
 CPR Pivot : 1437.3999999999999
 CPR TC  : 1436.5999999999997
 CPR BC : 1438.2
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : CUMMINSIND
 DateTime : 2023-02-15 12:15:00
 Close Price : 1610.8
 PrevWeek High : 1595.0 
 PrevWeek Low : 1433.5 
 CPR Pivot : 1437.3999

PE CONDITION TRIGGERED !
 Instrument : FEDERALBNK
 DateTime : 2023-02-07 15:15:00
 Close Price : 131.0
 PrevWeek High : 135.6 
 PrevWeek Low : 131.25 
 CPR Pivot : 135.75
 CPR TC  : 135.1
 CPR BC : 136.4 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : FEDERALBNK
 DateTime : 2023-02-08 09:15:00
 Close Price : 131.15
 PrevWeek High : 135.6 
 PrevWeek Low : 131.25 
 CPR Pivot : 135.75
 CPR TC  : 135.1
 CPR BC : 136.4 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : FEDERALBNK
 DateTime : 2023-02-08 10:15:00
 Close Price : 130.85
 PrevWeek High : 135.6 
 PrevWeek Low : 131.25 
 CPR Pivot : 135.75
 CPR TC  : 135.1
 CPR BC : 136.4 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : FEDERALBNK
 DateTime : 2023-02-08 11:15:00
 Close Price : 131.15
 PrevWeek High : 135.6 
 PrevWeek Low : 131.25 
 CPR Pivot : 135.75
 CPR TC  : 135.1
 CPR BC : 136.4 
------------------------------------------
PE CO

...

CE CONDITION TRIGGERED !
 Instrument : PFC
 DateTime : 2023-02-15 14:15:00
 Close Price : 148.9
 PrevWeek High : 145.7 
 PrevWeek Low : 142.0 
 CPR Pivot : 145.43333333333334
 CPR TC  : 143.44166666666666
 CPR BC : 147.425
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : PFC
 DateTime : 2023-02-15 15:15:00
 Close Price : 148.5
 PrevWeek High : 145.7 
 PrevWeek Low : 142.0 
 CPR Pivot : 145.43333333333334
 CPR TC  : 143.44166666666666
 CPR BC : 147.425
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : PFC
 DateTime : 2023-02-16 09:15:00
 Close Price : 149.15
 PrevWeek High : 145.7 
 PrevWeek Low : 142.0 
 CPR Pivot : 145.43333333333334
 CPR TC  : 143.44166666666666
 CPR BC : 147.425
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : PFC
 DateTime : 2023-02-16 10:15:00
 Close Price : 148.7
 PrevWeek High : 145.7 
 PrevWeek Low : 142.0 
 CPR Pivot : 145.43333333333334
 CPR TC  : 143.44166666666

PE CONDITION TRIGGERED !
 Instrument : RECLTD
 DateTime : 2023-02-16 15:15:00
 Close Price : 116.35
 PrevWeek High : 121.35 
 PrevWeek Low : 116.5 
 CPR Pivot : 120.01666666666667
 CPR TC  : 120.88333333333333
 CPR BC : 119.15 
------------------------------------------
------------   RELIANCE DETAILS ----------------------
PE CONDITION TRIGGERED !
 Instrument : RELIANCE
 DateTime : 2023-02-06 09:15:00
 Close Price : 2313.0
 PrevWeek High : 2366.6 
 PrevWeek Low : 2320.0 
 CPR Pivot : 2420.2833333333333
 CPR TC  : 2387.0666666666666
 CPR BC : 2453.5 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : RELIANCE
 DateTime : 2023-02-06 10:15:00
 Close Price : 2311.15
 PrevWeek High : 2366.6 
 PrevWeek Low : 2320.0 
 CPR Pivot : 2420.2833333333333
 CPR TC  : 2387.0666666666666
 CPR BC : 2453.5 
------------------------------------------
PE CONDITION TRIGGERED !
 Instrument : RELIANCE
 DateTime : 2023-02-06 11:15:00
 Close Price : 2310.8
 PrevWeek High : 2366.6

CE CONDITION TRIGGERED !
 Instrument : SRF
 DateTime : 2023-02-15 09:15:00
 Close Price : 2253.7
 PrevWeek High : 2229.65 
 PrevWeek Low : 2186.5 
 CPR Pivot : 2192.1833333333334
 CPR TC  : 2187.3166666666666
 CPR BC : 2197.05
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : SRF
 DateTime : 2023-02-15 10:15:00
 Close Price : 2253.5
 PrevWeek High : 2229.65 
 PrevWeek Low : 2186.5 
 CPR Pivot : 2192.1833333333334
 CPR TC  : 2187.3166666666666
 CPR BC : 2197.05
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : SRF
 DateTime : 2023-02-15 11:15:00
 Close Price : 2256.65
 PrevWeek High : 2229.65 
 PrevWeek Low : 2186.5 
 CPR Pivot : 2192.1833333333334
 CPR TC  : 2187.3166666666666
 CPR BC : 2197.05
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : SRF
 DateTime : 2023-02-15 12:15:00
 Close Price : 2247.2
 PrevWeek High : 2229.65 
 PrevWeek Low : 2186.5 
 CPR Pivot : 2192.1833333333334
 CPR TC  :

CE CONDITION TRIGGERED !
 Instrument : TCS
 DateTime : 2023-02-16 09:15:00
 Close Price : 3558.05
 PrevWeek High : 3542.0 
 PrevWeek Low : 3446.95 
 CPR Pivot : 3339.0499999999997
 CPR TC  : 3348.8749999999995
 CPR BC : 3329.225
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : TCS
 DateTime : 2023-02-16 10:15:00
 Close Price : 3570.0
 PrevWeek High : 3542.0 
 PrevWeek Low : 3446.95 
 CPR Pivot : 3339.0499999999997
 CPR TC  : 3348.8749999999995
 CPR BC : 3329.225
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : TCS
 DateTime : 2023-02-16 11:15:00
 Close Price : 3562.25
 PrevWeek High : 3542.0 
 PrevWeek Low : 3446.95 
 CPR Pivot : 3339.0499999999997
 CPR TC  : 3348.8749999999995
 CPR BC : 3329.225
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : TCS
 DateTime : 2023-02-16 12:15:00
 Close Price : 3559.9
 PrevWeek High : 3542.0 
 PrevWeek Low : 3446.95 
 CPR Pivot : 3339.0499999999997
 CPR T

CE CONDITION TRIGGERED !
 Instrument : VOLTAS
 DateTime : 2023-02-13 14:15:00
 Close Price : 850.3
 PrevWeek High : 828.1 
 PrevWeek Low : 811.35 
 CPR Pivot : 791.2833333333333
 CPR TC  : 796.9666666666666
 CPR BC : 785.6
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : VOLTAS
 DateTime : 2023-02-13 15:15:00
 Close Price : 845.7
 PrevWeek High : 828.1 
 PrevWeek Low : 811.35 
 CPR Pivot : 791.2833333333333
 CPR TC  : 796.9666666666666
 CPR BC : 785.6
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : VOLTAS
 DateTime : 2023-02-14 09:15:00
 Close Price : 868.3
 PrevWeek High : 828.1 
 PrevWeek Low : 811.35 
 CPR Pivot : 791.2833333333333
 CPR TC  : 796.9666666666666
 CPR BC : 785.6
------------------------------------------
CE CONDITION TRIGGERED !
 Instrument : VOLTAS
 DateTime : 2023-02-14 10:15:00
 Close Price : 859.75
 PrevWeek High : 828.1 
 PrevWeek Low : 811.35 
 CPR Pivot : 791.2833333333333
 CPR TC  : 796.96666666